# PlotyDash

In [34]:

#import required libraries
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px

#importing the dataset
spacex_df= pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload= spacex_df["Payload Mass (kg)"].max()
min_payload= spacex_df["Payload Mass (kg)"].min()

JupyterDash.infer_jupyter_proxy_config()
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

#creating app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                style={'textAlign': 'center', 'color':'#503D36', 'fontsize': 40}),
                dcc.Dropdown(id= 'site_dropdown',
                            options =[
                                {'label': 'All Sites', 'value': 'ALL'},
                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                            
                                ],
                                value='All',
                                placeholder='Select a Launch Site here',
                                searchable=True
                                ),

                html.Br(),

                       
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                             
                                dcc.RangeSlider(
                                    id='payload_slider',
                                    min=0,
                                    max=10000,
                                    step=1000,  
                                    value=[min_payload,max_payload]
                                ),
                            
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                              
                                ])

#creating callback_1
@app.callback(
     dash.dependencies.Output(component_id='success-pie-chart',component_property='figure'),
     [dash.dependencies.Input(component_id='site_dropdown',component_property='value')]               
)
#to get pie chart
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'All':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches By Site')
        return fig
    else:
        filtered_df=spacex_df[spacex_df['Launch Site']== entered_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df,values='class count',names='class',title=f" Total Success Launches for site {entered_site}")
        return fig

#creating callback_2
@app.callback(
     dash.dependencies.Output(component_id='success-payload-scatter-chart',component_property='figure'),
     [dash.dependencies.Input(component_id='site_dropdown',component_property='value'),
      dash.dependencies.Input(component_id="payload_slider", component_property="value")]               
)
#to get scattergraph
def get_scattergraph(entered_site,payload_slider):
    if entered_site == 'All':
        low, high = payload_slider
        df= spacex_df
        filter_df = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(df[filter_df], x="Payload Mass (kg)", y="class", color="Booster Version", size='Payload Mass (kg)')
    else:
        low, high = payload_slider
        df  = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        filter_df  = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(df[filter_df], x="Payload Mass (kg)", y="class", color="Booster Version", size='Payload Mass (kg)')
    return fig

#run the app
if __name__ == '__main__':
    app.run_server()

Dash app running on https://hub.gke2.mybinder.org/user/plotly-jupyter-dash-t0y8nv3l/proxy/8050/
